In [12]:
import tensorflow as tf
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
M = 10
N = 2*M+1
k = 0.5
h = 0.5
tfM = tf.constant(M)
tfN = tf.constant(N)
tfk = tf.constant(k)
tfh = tf.constant(h)

In [14]:
tfgrid = tf.multiply(tfk,tf.cast(
        tf.range(tf.multiply(tfM,tf.constant(-1)),tf.add(tfM,tf.constant(1))),
        tf.float32))

In [15]:
sess.run(tfgrid)

array([-5. , -4.5, -4. , -3.5, -3. , -2.5, -2. , -1.5, -1. , -0.5,  0. ,
        0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ], dtype=float32)

In [22]:
def tff(theta, x):
    fval = (theta[0])*(theta[1] - x)
    return fval

def tfg(theta, x):
    gval = theta[2]*tf.ones(tf.shape(x))
    return gval

In [17]:
# quick code to get automatic gradient
# note that this is *summed* over all entries of the x=tfgrid tensor
th = tf.placeholder("float",[3])
thval = [0.5,0.7,0.2]
sess.run(tf.gradients(tff(theta=th,x=tfgrid),th), feed_dict = {th: thval})
sess.run(tf.gradients(tfg(theta=th,x=tfgrid),th), feed_dict = {th: thval})

[array([  0.,   0.,  21.], dtype=float32)]

In [18]:
import numpy as np

In [19]:
# let's build up the propagator in pure tensorflow
tfgridx, tfgridy = tf.meshgrid(tfgrid, tfgrid, indexing='ij')
tfmu = tf.add(tfgridy,tf.multiply(tff(theta=th,x=tfgridy),tfh))
tfsig = tf.multiply(tf.sqrt(tfh),tfg(theta=th,x=tfgridy))
tfc0 = tf.reciprocal(tf.multiply(tf.sqrt(tf.multiply(2.0,tf.constant(np.pi))),tfsig))
tfnumer = tf.multiply(-1.0,tf.square(tf.subtract(tfgridx,tfmu)))
tfdenom = tf.multiply(2.0,tf.square(tfsig))
tfprop = tf.multiply(tfc0,tf.exp(tf.divide(tfnumer,tfdenom)))

In [20]:
np.mean(np.abs(sess.run(tf.reduce_sum(tfprop,axis=0), feed_dict = {th: thval})*k  - 1.0))

0.28572911

In [21]:
sess.run(tf.gradients(tfprop,th), feed_dict = {th: thval})

[array([ 18.22197533,   2.13901258,  -7.51260948], dtype=float32)]